https://github.com/Pranav082001/stock-analyzer-bot/blob/master/stock_analyzer_bot.ipynb

In [2]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent
warnings.filterwarnings("ignore")

In [3]:
os.environ["OPENAI_API_KEY"] = "sk-xxxxx"

In [4]:
llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

### Defining the custom Tools

In [6]:
import yfinance as yf

# Fetch stock data from Yahoo Finance

def get_stock_price(ticker,history=5):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    # print(df.columns)
    
    return df.to_string()

print(get_stock_price("KOTAKBANK.NS"))

                  Close   Volume
Date                            
2023-11-20  1770.550049  1923651
2023-11-21  1767.199951  2705765
2023-11-22  1745.650024  4513732
2023-11-23  1739.300049  2836289
2023-11-24  1744.000000  2561979


In [8]:
# Script to scrap top5 googgle news for given company name

def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    # time.sleep(4) #To avoid rate limit error
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Axis Bank"))

Recent News:

0. Axis Bank Analyst Day: Key takeaways, share price targets & more
1. Axis Bank analyst meet: To focus on strong digi-retail banking; analysts 
bullish on stock amid reasonable valuations | Mint



In [9]:
# Fetch financial statements from Yahoo Finance

def get_financial_statements(ticker):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    
    # cash_flow = company.cash_flow.to_string()
    # print(balance_sheet)
    # print(cash_flow)
    return balance_sheet
print(get_financial_statements("TATAPOWER.NS"))

                                                                           2023-03-31       2022-03-31      2021-03-31
Ordinary Shares Number                                                   3195339547.0     3196991847.0    3195339547.0
Share Issued                                                             3195339547.0     3196991847.0    3195339547.0
Net Debt                                                               447846700000.0   445128500000.0  393856600000.0
Total Debt                                                             529230000000.0   511952100000.0  467054800000.0
Tangible Book Value                                                    255477800000.0   192170700000.0  191818400000.0
Invested Capital                                                       777618600000.0   700316500000.0  654904300000.0
Working Capital                                                       -130012100000.0  -141013900000.0 -114893300000.0
Net Tangible Assets                             

In [10]:
from langchain.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()

search("Stock news India")

ImportError: Could not import duckduckgo-search python package. Please install it with `pip install duckduckgo-search`.

In [23]:
# Making tool list

tools=[
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    # Tool(
    #     # name="DuckDuckGo Search",
    #     # func=search.run,
    #     # description="Use only when you need to get NSE/BSE stock ticker from internet, you can also get recent stock related news. Dont use it for any other analysis or task"
    # ),
    Tool(
        name="get recent news",
        func=get_recent_stock_news,
        description="Use this to fetch recent news about stocks"
    ),

    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it"
    ) 


]

#### Approach1- React Agent

In [24]:
from langchain.agents import initialize_agent 

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [25]:
print(zero_shot_agent.agent.llm_chain.prompt.template) 

Answer the following questions as best you can. You have access to the following tools:

get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it 
get recent news: Use this to fetch recent news about stocks
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get stock data, get recent news, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
T

In [18]:
#Adding predefine evaluation steps in the agent Prompt

new_prompt="""You are a financial advisor. Give stock recommendations for given query.
Everytime first you should identify the company name and get the stock ticker symbole for indian stock.
Answer the following questions as best you can. You have access to the following tools:

get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it 
get recent news: Use this to fetch recent news about stocks
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it

steps- 
Note- if you fail in satisfying any of the step below, Just move to next one
1) Get the company name and search for the "company name + NSE/BSE stock ticker" on internet. Dont hallucinate extract stock ticker as it is from the text. Output- stock ticker
2) Use "get stock data" tool to gather stock info. Output- Stock data
3) Get company's historic financial data using "get financial statements". Output- Financial statement
4) Analyze the stock based on gathered data and give detail analysis for investment choice. provide numbers and reasons to justify your answer. Output- Detailed stock Analysis

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [19]:
#Setting up new prompt
zero_shot_agent.agent.llm_chain.prompt.template=new_prompt

In [20]:
zero_shot_agent("Is Bajaj Finance a good investment choice right now?")



> Entering new AgentExecutor chain...
I need to find the stock ticker symbol for Bajaj Finance and gather stock data and financial statements to analyze its performance.
Action Input: Search for "Bajaj Finance NSE stock ticker" on the internet
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Action: Search for "Bajaj Finance NSE stock ticker" on the internet
Action Input: Bajaj Finance NSE stock ticker
Observation: Search for "Bajaj Finance NSE stock ticker" on the internet is not a valid tool, try one of [get stock data, get recent news, get financial statements].
Thought:Action: Use "get stock data" tool to gather stock info
Action Input: Bajaj Finance stock ticker
Observation: Use "get stock data" tool to gather stock info is not a valid tool, try one of [get stock data, get recent news, get financial statements].
Thought:Action: Use "get stock data" tool to gather stock info
Action Input: BAJFINANCE.NS
Observation: Use "get stock data" tool to gather stock i

KeyboardInterrupt: 

### Approach 2

In [27]:
import openai
from langchain.llms import OpenAI

llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

In [28]:

#Openai function calling

import json
function=[
        {
        "name": "get_company_Stock_ticker",
        "description": "This will get the indian NSE/BSE stock ticker of the company",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "This is the stock symbol of the company.",
                },

                "company_name": {
                    "type": "string",
                    "description": "This is the name of the company given in query",
                }
            },
            "required": ["company_name","ticker_symbol"],
        },
    }
]



def get_stock_ticker(query):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[{
                "role":"user",
                "content":f"Given the user request, what is the comapany name and the company stock ticker ?: {query}?"
            }],
            functions=function,
            function_call={"name": "get_company_Stock_ticker"},
    )
    message = response["choices"][0]["message"]
    arguments = json.loads(message["function_call"]["arguments"])
    company_name = arguments["company_name"]
    company_ticker = arguments["ticker_symbol"]
    return company_name,company_ticker

get_stock_ticker("How is ths stock of Paytm doing?")

('Paytm', 'PAYTM')

In [29]:
def Anazlyze_stock(query):
    #agent.run(query) Outputs Company name, Ticker
    Company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(Company_name)

    # available_information=f"Stock Price: {stock_data}\n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
    available_information=f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

    print("\n\nAnalyzing.....\n")
    analysis=llm(f"Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} "
             )
    print(analysis)

    return analysis

In [30]:
out=Anazlyze_stock("Shall I invest in Adani power right now?")  

{'Query': 'Shall I invest in Adani power right now?', 'Company_name': 'Adani Power', 'Ticker': 'ADANIPOWER'}


Analyzing.....

Based on the available data and recent news, here is a detailed stock analysis of Adani Power:

1. Financials: Adani Power has shown consistent growth in its financials over the past three years. The company's total equity, tangible book value, and invested capital have all increased steadily. However, there has been a negative trend in retained earnings, which is a cause for concern.

2. Debt: Adani Power has a significant amount of debt, with net debt and total debt increasing over the years. This indicates that the company relies heavily on borrowed funds to finance its operations. Investors should consider the impact of this debt on the company's financial stability and ability to generate profits.

3. Working Capital: Adani Power has shown a mixed trend in working capital. While it had a positive working capital in 2023, it had negative working capital in 

In [31]:
Anazlyze_stock("Is it a good time to invest in Yes Bank?")   

{'Query': 'Is it a good time to invest in Yes Bank?', 'Company_name': 'Yes Bank', 'Ticker': 'YESBANK'}


Analyzing.....

1. Financial Stability: Yes Bank has shown improvement in its financials over the past three years. The total equity, common stock equity, and net tangible assets have increased consistently, indicating a stronger financial position.

2. Debt Levels: The company's net debt and total debt have increased over the years, which could be a concern. However, the increase in debt is accompanied by an increase in invested capital, suggesting that the company is utilizing the borrowed funds for growth and expansion.

3. Investments and Advances: Yes Bank has made significant investments and advances, which indicates its focus on diversifying its revenue streams and generating additional income. This could potentially lead to higher profitability in the future.

4. Cash and Cash Equivalents: The company has a substantial amount of cash and cash equivalents, which provides a cu

"1. Financial Stability: Yes Bank has shown improvement in its financials over the past three years. The total equity, common stock equity, and net tangible assets have increased consistently, indicating a stronger financial position.\n\n2. Debt Levels: The company's net debt and total debt have increased over the years, which could be a concern. However, the increase in debt is accompanied by an increase in invested capital, suggesting that the company is utilizing the borrowed funds for growth and expansion.\n\n3. Investments and Advances: Yes Bank has made significant investments and advances, which indicates its focus on diversifying its revenue streams and generating additional income. This could potentially lead to higher profitability in the future.\n\n4. Cash and Cash Equivalents: The company has a substantial amount of cash and cash equivalents, which provides a cushion for any unforeseen events or financial obligations. This liquidity can also be utilized for future investmen

In [32]:
Anazlyze_stock("How vednata share is doing?")    

{'Query': 'How vednata share is doing?', 'Company_name': 'Vedanta', 'Ticker': 'VEDL'}


Analyzing.....

1. The number of ordinary shares for Vedanta has remained relatively stable over the past three years, indicating no significant dilution or increase in share capital.
2. The net debt of Vedanta has increased significantly from 2021 to 2023, indicating a higher level of leverage and potential financial risk for the company.
3. The tangible book value of Vedanta has decreased from 2021 to 2022, but then increased in 2023. This indicates that the company's assets are primarily composed of tangible assets such as property, plant, and equipment.
4. The working capital of Vedanta has fluctuated over the past three years, with a negative value in 2021 and a positive value in 2022 and 2023. This indicates that the company may have had difficulty in meeting its short-term obligations in 2021 but has improved its liquidity position in the subsequent years.
5. The total equity of Vedanta has i

"1. The number of ordinary shares for Vedanta has remained relatively stable over the past three years, indicating no significant dilution or increase in share capital.\n2. The net debt of Vedanta has increased significantly from 2021 to 2023, indicating a higher level of leverage and potential financial risk for the company.\n3. The tangible book value of Vedanta has decreased from 2021 to 2022, but then increased in 2023. This indicates that the company's assets are primarily composed of tangible assets such as property, plant, and equipment.\n4. The working capital of Vedanta has fluctuated over the past three years, with a negative value in 2021 and a positive value in 2022 and 2023. This indicates that the company may have had difficulty in meeting its short-term obligations in 2021 but has improved its liquidity position in the subsequent years.\n5. The total equity of Vedanta has increased from 2021 to 2023, indicating a positive growth in the company's net worth.\n6. The retain